In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn import preprocessing

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers.convolutional import Conv3D, MaxPooling3D, Conv2D, AveragePooling2D, AveragePooling3D
from keras.layers import Dense, GlobalAveragePooling3D, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.optimizers import SGD, RMSprop, Adadelta
from keras.utils import np_utils, generic_utils
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional_recurrent import ConvLSTM2D

In [ ]:
# image specification
img_rows, img_cols = 125, 57

# training data
X_tr = []      # variable to store the entire dataset

In [ ]:
from tqdm import tqdm 
from google.colab.patches import cv2_imshow
sub1_t1a1 = os.path.join("/content/drive/My Drive/", "Subject1Activity1Trial1Camera1")
listing = os.listdir(sub1_t1a1)

frames = []
i = 0
for image in tqdm(listing):
  if i<150:
    img_path = os.path.join(sub1_t1a1, image)
    img = cv2.imread(img_path)
    img = cv2.resize(img, (img_rows, img_cols), interpolation = cv2.INTER_AREA)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    frames.append(img)
    i=i+1
  else:
    break

input_img = np.array(frames)
print(input_img.shape)
ipt = np.rollaxis(np.rollaxis(input_img, 2, 0), 2, 0)
print(ipt.shape)
ipt = np.rollaxis(ipt, 2, 0)
print(ipt.shape)
X_tr.append(ipt)
print(len(X_tr))

 74%|███████▍  | 150/202 [03:12<01:12,  1.40s/it]

(150, 57, 125, 3)
(57, 125, 150, 3)
(150, 57, 125, 3)
1


In [ ]:
print(len(ipt))

150


In [ ]:
sub1_t1a2 = os.path.join("/content/drive/My Drive/", "Subject1Activity2Trial1Camera1")
listing = os.listdir(sub1_t1a2)

frames = []
i = 0
for image in tqdm(listing):
  if i < 150:
    img_path = os.path.join(sub1_t1a2, image)
    img = cv2.imread(img_path)
    img = cv2.resize(img, (img_rows, img_cols), interpolation = cv2.INTER_AREA)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    frames.append(img)
    i=i+1
  else:
    break

input_img = np.array(frames)
print(input_img.shape)
ipt = np.rollaxis(np.rollaxis(input_img, 2, 0), 2, 0)
print(ipt.shape)
ipt = np.rollaxis(ipt, 2, 0)
print(ipt.shape)
X_tr.append(ipt)
print(len(X_tr))

In [ ]:
X_tr_array = np.array(X_tr)  # convert frames read into array

num_samples = len(X_tr_array)
print(num_samples)

2


In [ ]:
label = np.ones((num_samples, ), dtype = int)
label[0] = 0
label[1] = 1

In [ ]:
img_depth = 150
train_data = [X_tr_array, label]

(X_train, y_train) = (train_data[0], train_data[1])
print('X_train shape: ', X_train.shape)

train_set = np.zeros((num_samples, img_depth, img_cols, img_rows, 3))

for h in range(num_samples):
  train_set[h][:][:][:] = X_train[h, :, :, :]

patch_size = 150  # no. of frames used for each video

print(train_set.shape, 'train_samples')

X_train shape:  (2, 150, 57, 125, 3)
(2, 150, 57, 125, 3) train_samples


In [ ]:
nb_classes = 2
Y_train = np_utils.to_categorical(y_train, nb_classes)

In [ ]:
# pre-processing

train_set = train_set.astype('float32')
print(np.mean(train_set))
train_set -= np.mean(train_set)
print(np.max(train_set))
train_set /= np.max(train_set)

133.84485
121.15515


In [ ]:
# low resolution network
weight_decay = 0.005
from keras import regularizers
model = Sequential()
model.add(MaxPooling3D(pool_size = (1,2,2), input_shape=(patch_size, 57, 125, 3)))
model.add(Conv3D(8, (5,5,5), activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(MaxPooling3D(pool_size=(1,2,2)))
model.add(Conv3D(32, (3,5,5), activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Conv3D(64,(3,3,5), activation='relu',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(MaxPooling3D(pool_size=(1, 1, 4 )))
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Dropout(0.5))

model.add(Dense(nb_classes,kernel_initializer='normal',kernel_regularizer=regularizers.l2(weight_decay)))

model.add(Activation('softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
max_pooling3d (MaxPooling3D) (None, 150, 28, 62, 3)    0         
_________________________________________________________________
conv3d (Conv3D)              (None, 146, 24, 58, 8)    3008      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 146, 12, 29, 8)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 144, 8, 25, 32)    19232     
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 72, 4, 12, 32)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 70, 2, 8, 64)      92224     
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 70, 2, 2, 64)      0

In [ ]:
import os
save_dir = os.path.join(os.getcwd(), 'saved_model')
model_name = '3D_CNN_UP_Fall_ver1'
if not os.path.isdir(save_dir):
  os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
#checkpoint = ModelCheckpoint(model_path, monitor='val_acc',
#                             save_best_only=True, verbose=1)
#early_stop = EarlyStopping(monitor='val_loss', patience=50, verbose=1)
#lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.05, 
#                              cooldown=0, patience=10, min_lr=0.005/(2^4),verbose=1)

In [ ]:
sgd = SGD(lr=0.005,  momentum=0.9, nesterov=False)
rms = RMSprop(decay=1e-6)
ada = Adadelta(lr=0.1,decay=1e-6)
model.compile(loss='categorical_crossentropy', 
              optimizer=sgd,
              #optimizer=ada,
              metrics=['acc'])

In [ ]:
model.fit(train_set, Y_train, epochs = 5)

Epoch 1/5
1/1 [==============================] - 0s 2ms/step - loss: 7.8390 - acc: 0.5000
Epoch 2/5
1/1 [==============================] - 0s 1ms/step - loss: 7.7191 - acc: 0.5000
Epoch 3/5
1/1 [==============================] - 0s 1ms/step - loss: 7.7090 - acc: 0.0000e+00
Epoch 4/5
1/1 [==============================] - 0s 1ms/step - loss: 7.6268 - acc: 0.5000
Epoch 5/5
1/1 [==============================] - 0s 1ms/step - loss: 7.7387 - acc: 0.0000e+00
